# Consulta de API y Estructuración de Datos

Este notebook muestra cómo usar `requests` para consultar una API y `pandas` para estructurar y analizar los datos obtenidos.


In [1]:
import requests
import pandas as pd
import json
from datetime import datetime


## 1. Autenticación en la API

Realizamos el login para obtener el token de autenticación que usaremos en las siguientes solicitudes.


In [2]:
url_login = "http://localhost:8080/api/auth/login"

credenciales = {
    "identification": "133553",
    "password": "Nutabe*2020"
}

response = requests.post(url_login, json=credenciales)

if response.status_code == 200:
    data_response = response.json()
    token = data_response.get('token') or data_response.get('accessToken') or data_response.get('access_token')
    
    if token:
        print(f"Autenticación exitosa - Status Code: {response.status_code}")
        AUTH_TOKEN = token
        token_type = data_response.get('tokenType', 'Bearer') or 'Bearer'
        AUTH_TOKEN_TYPE = token_type
    else:
        print("Error: No se pudo obtener el token de la respuesta")
else:
    print(f"Error en la autenticación - Status Code: {response.status_code}")


Autenticación exitosa - Status Code: 200


In [3]:
def get_auth_headers():
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'{AUTH_TOKEN_TYPE} {AUTH_TOKEN}'
    }
    return headers


## 2. Consulta de documentos fiscales

Realizamos una petición al endpoint de documentos fiscales usando el token de autenticación.


In [4]:
url_fiscal_documents = "http://localhost:8080/api/fiscal-documents"
headers = get_auth_headers()

response = requests.get(url_fiscal_documents, headers=headers)

if response.status_code == 200:
    fiscal_documents_data = response.json()
    print(f"Petición exitosa - Status Code: {response.status_code} - Registros obtenidos: {len(fiscal_documents_data)}")
else:
    print(f"Error en la petición - Status Code: {response.status_code}")


Petición exitosa - Status Code: 200 - Registros obtenidos: 79879


## 3. Crear DataFrame con los datos

Convertimos los datos JSON obtenidos de la API en un DataFrame de pandas para facilitar su análisis y manipulación.


In [5]:
if 'fiscal_documents_data' in globals() and fiscal_documents_data:
    df_fiscal_documents = pd.DataFrame(fiscal_documents_data)
    print(f"DataFrame creado - Registros: {len(df_fiscal_documents)}")
else:
    print("Error: No hay datos disponibles para crear el DataFrame")


DataFrame creado - Registros: 79879
